## Imports

In [1]:
import csv
import obspy
from obspy import signal
import obspy.signal.filter
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import random
import pandas as pd
from joblib import dump, load
import os
from datetime import datetime

In [3]:
CONTEXT_WINDOW = 90
SAMPLES = 7201

## Positive Examples

In [4]:
#get all events
def get_full_event_list(path = "../Data/allevents.csv"):
    all = pd.read_csv(path)
    keep = ['year', 'day', 'date', 'oTime', 'spTime', 'distance', 'duration', 'quality']
    events = all[keep]
    events = events[events['year']> 1999]
    return events[4:]

In [5]:
def slice_and_filter(time, stream):
    stream = stream.slice(starttime=time, endtime=time+CONTEXT_WINDOW)

    #filtered_data = obspy.signal.filter.highpass(stream, freq=2.5, df=5.0001)

    #stream.data = filtered_data
    return stream

In [6]:
def get_positive_data(path, event_list):
    clean_data = []
    
    for _, row in event_list.iterrows():
        HHE = path+f"{row['year']}/MN/WDD/HHE.D/MN.WDD..HHE.D.{row['year']}.{str(row['day']).zfill(3)}"
        HHN = path+f"{row['year']}/MN/WDD/HHN.D/MN.WDD..HHN.D.{row['year']}.{str(row['day']).zfill(3)}"
        HHZ = path+f"{row['year']}/MN/WDD/HHZ.D/MN.WDD..HHZ.D.{row['year']}.{str(row['day']).zfill(3)}"
        
        try:    
            st = obspy.read(HHE)
            st += obspy.read(HHN)
            st += obspy.read(HHZ)
            quakeTime = obspy.UTCDateTime(row['date']+row['oTime'])
            st = slice_and_filter(quakeTime, st)
            if len(st) == 3 and len(st[0]) == SAMPLES and len(st[1]) == SAMPLES and len(st[2]) == SAMPLES:
                clean_data.append(st)
    
        except Exception as e:
            print(f"Missing File {row['year']}, {row['day']}")
            continue

    return clean_data

In [7]:
def make_utc_list(events):
    utc_times = []
    for _, row in events.iterrows():
        utc_times.append(obspy.UTCDateTime(row['date']+row['oTime']))
    return utc_times

In [8]:
def create_neg_examples(path, utc_times, amount, years):
    # Initialize an empty list to store the negative examples
    negative_examples = []

    # Loop to generate random negative examples
    while len(negative_examples) < amount:

        ran_day = random.randint(1, 365)
        ran_hr = random.randint(0, 23)
        ran_min = random.randint(0, 59)
        ran_sec = random.randint(0, 59)
        ran_year = random.choice(years)

        if ran_day == 97 and ran_year == 2010:
            continue

        # Generate a random UTC time on the selected day
        random_utc_time = obspy.UTCDateTime(year=ran_year, julday=ran_day, hour=ran_hr, minute=ran_min, second=ran_sec)

        flag = False

        for time in utc_times:
            #if the window being created is in an earthquake window
            if random_utc_time < time + CONTEXT_WINDOW and random_utc_time > time:
                flag = True

        if not flag:
            HHE = path+f"{ran_year}/MN/WDD/HHE.D/MN.WDD..HHE.D.{ran_year}.{str(ran_day).zfill(3)}"
            HHN = path+f"{ran_year}/MN/WDD/HHN.D/MN.WDD..HHN.D.{ran_year}.{str(ran_day).zfill(3)}"
            HHZ = path+f"{ran_year}/MN/WDD/HHZ.D/MN.WDD..HHZ.D.{ran_year}.{str(ran_day).zfill(3)}"

            try:
                st = obspy.read(HHE)
                st += obspy.read(HHN)
                st += obspy.read(HHZ)

                st = slice_and_filter(random_utc_time, st)
                if len(st) == 3 and len(st[0]) == SAMPLES and len(st[1]) == SAMPLES and len(st[2]) == SAMPLES:
                    negative_examples.append(st)

            except Exception as e:
                continue
    return negative_examples

# Run The Above

In [9]:
#define all years
years = list(range(2000, 2015))
#get whole event list
all_events = get_full_event_list('../Data/allevents.csv')

local_events = all_events[all_events['spTime'] <= 20]
distant_events = all_events[all_events['spTime'] > 20]

#get +ve examples
local_earthquake_data = np.array(get_positive_data('../Data/', local_events))
#generate +ve labels
local_earthquake_labels = np.asarray([1] * len(local_earthquake_data))

distant_earthquake_data = np.array(get_positive_data('../Data/', distant_events))
#generate +ve labels
distant_earthquake_labels = np.asarray([2] * len(distant_earthquake_data))

utc_times = make_utc_list(all_events)

neg_examples = create_neg_examples('../Data/', utc_times, len(distant_earthquake_labels)+len(local_earthquake_labels), years)
negative_labels = np.asarray([0] * len(neg_examples))

combined_data = np.concatenate((local_earthquake_data, distant_earthquake_data, neg_examples), axis=0)
combined_labels = np.concatenate((local_earthquake_labels, distant_earthquake_labels, negative_labels), axis=0)

Missing File 2003, 124
Missing File 2002, 355
Missing File 2002, 229
Missing File 2002, 205
Missing File 2001, 290
Missing File 2001, 242
Missing File 2001, 242
Missing File 2001, 240
Missing File 2001, 231
Missing File 2001, 231
Missing File 2001, 230
Missing File 2001, 226
Missing File 2001, 71
Missing File 2001, 15
Missing File 2000, 261
Missing File 2000, 204
Missing File 2000, 129
Missing File 2000, 126
Missing File 2003, 245
Missing File 2002, 354
Missing File 2002, 328
Missing File 2002, 305
Missing File 2002, 194
Missing File 2001, 273
Missing File 2001, 245
Missing File 2001, 242
Missing File 2001, 229
Missing File 2000, 244
Missing File 2000, 239


FFT

In [40]:
def apply_fft(data):
    fft_data = np.fft.fft(data)
    return np.abs(fft_data)

In [39]:
# Apply FFT to combined_data
combined_data_fft = [apply_fft(sample) for sample in combined_data]

NameError: name 'combined_data' is not defined

Dump Data

In [11]:
dump(combined_data, '../DataDumps/rawishdata.joblib')

['../DataDumps/rawishdata.joblib']

Dump 2-Class

In [57]:
#combined_labels[combined_labels == 2] = 1
# Save the combined_data and combined_labels
dump(np.array(combined_data_fft), '../DataDumps/all_data_fft.joblib')
dump(combined_labels, '../DataDumps/2class_labels.joblib')

['../DataDumps/2class_labels.joblib']

Dump 3-Class

In [33]:
def balance_data(data, labels):
    # Find the indices corresponding to each label
    indices_0 = np.where(labels == 0)[0]
    indices_1 = np.where(labels == 1)[0]
    indices_2 = np.where(labels == 2)[0]
    
    # Find the minimum number of samples among all labels
    min_samples = min(len(indices_0), len(indices_1), len(indices_2))
    
    # Randomly select min_samples for each label
    selected_indices_0 = np.random.choice(indices_0, min_samples, replace=False)
    selected_indices_1 = np.random.choice(indices_1, min_samples, replace=False)
    selected_indices_2 = np.random.choice(indices_2, min_samples, replace=False)
    
    # Concatenate the selected indices
    selected_indices = np.concatenate([selected_indices_0, selected_indices_1, selected_indices_2])
    
    # Extract the balanced data and labels
    balanced_data = data[selected_indices]
    balanced_labels = labels[selected_indices]
    
    return balanced_data, balanced_labels

In [28]:
raw = load('../DataDumps/rawishdata.joblib')
labels = load('../DataDumps/all_labels.joblib')

In [46]:
balanced_data, balanced_labels = balance_data(raw, labels)
# Save the combined_data and combined_labels
dump(np.array(balanced_data), '../DataDumps/3classraw.joblib')
dump(balanced_labels, '../DataDumps/3class_labels.joblib')

['../DataDumps/3class_labels.joblib']

In [43]:
balanced_data_fft = [apply_fft(sample) for sample in balanced_data]


In [47]:
dump(np.array(balanced_data_fft), '../DataDumps/3classfft.joblib')

['../DataDumps/3classfft.joblib']

# Spectral Subtraction

In [46]:
neg = combined_data_fft[1037:]

In [47]:
avg_noise_spectrum = np.mean(neg, axis=0)

In [48]:
# Define function for spectral subtraction
def spectral_subtraction(earthquake_data_fft, avg_noise_spectrum):
    # Subtract noise spectrum from earthquake data spectrum
    cleaned_data_fft = earthquake_data_fft - avg_noise_spectrum
    return cleaned_data_fft

# Apply spectral subtraction to earthquake data
cleaned_data_fft = [spectral_subtraction(sample, avg_noise_spectrum) for sample in combined_data_fft]

# Dump cleaned data to file


In [49]:
dump(np.array(cleaned_data_fft), '../DataDumps/SpectralSubt.joblib')

['../DataDumps/SpectralSubt.joblib']